In [1]:
import os
# Find the latest version of spark 2.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.0'
spark_version = 'spark-3.0.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:13 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Fetched 252 kB in 2s (143 kB/s)
Reading package lists... Done


In [2]:
#Start SparkSession

from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("BigData-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [3]:
# Read in data into DataFrame

from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Digital_Music_Purchase_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|    product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   10293599|R14LVB34Z2Z53I|B000YMOQZY|     262935067|Knockin' On Heave...|Digital_Music_Pur...|          5|            0|          0|   N|                Y|           favorites|Great  rendition....| 2015-08-31|
|         US|    6087195|R2M0R9CWPC82LT|B00ISAEC80|     452318038|            Flawless|Digital_Music_Pur

In [4]:
# Filter and Create a new DataFrame to retrieve all the rows where total_votes count is equal to or greater than 20

vote_count20 = df.filter("total_votes >= 20")
vote_count20.show()

+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|    product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   21585599|R1S0N56H5B9SUX|B00ZQKVD4Q|     550273770|        Live in Cuba|Digital_Music_Pur...|          5|           30|         30|   N|                Y|Excellent Jazz album|Excellent album. ...| 2015-08-31|
|         US|   16227164| RCDI7Z8VCZJI9|B00YHTO1HS|     771772746|Badlands (Deluxe)...|Digital_Music_Pur

In [5]:
# Create a DataFrame to retrieve all the rows that have more than 20 total votes
# and where helpful_votes divided by total_votes is equal to or greater than 50%.

HelpOverTotal = vote_count20.filter(vote_count20["helpful_votes"]/vote_count20["total_votes"]>=0.5)
HelpOverTotal.show()

+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|    product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   21585599|R1S0N56H5B9SUX|B00ZQKVD4Q|     550273770|        Live in Cuba|Digital_Music_Pur...|          5|           30|         30|   N|                Y|Excellent Jazz album|Excellent album. ...| 2015-08-31|
|         US|   16227164| RCDI7Z8VCZJI9|B00YHTO1HS|     771772746|Badlands (Deluxe)...|Digital_Music_Pur

In [6]:
#Create a DataFrame that retrieves all the rows where there was atleast 20 votes, at least 50% helpful and
# a review was written as part of the Vine program (paid), vine == 'Y'

VineTableY = HelpOverTotal.filter(HelpOverTotal["vine"] == "Y")
VineTableY.show()

+-----------+-----------+---------+----------+--------------+-------------+----------------+-----------+-------------+-----------+----+-----------------+---------------+-----------+-----------+
|marketplace|customer_id|review_id|product_id|product_parent|product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|review_headline|review_body|review_date|
+-----------+-----------+---------+----------+--------------+-------------+----------------+-----------+-------------+-----------+----+-----------------+---------------+-----------+-----------+
+-----------+-----------+---------+----------+--------------+-------------+----------------+-----------+-------------+-----------+----+-----------------+---------------+-----------+-----------+



In [7]:
#Create a DataFrame that retrieves all the rows where there was atleast 20 votes, at least 50% helpful and
# a review was written as part of the Vine program (unpaid), vine == 'N'

VineTableN = HelpOverTotal.filter(HelpOverTotal["vine"] == "N")
VineTableN.show()

+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|    product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   21585599|R1S0N56H5B9SUX|B00ZQKVD4Q|     550273770|        Live in Cuba|Digital_Music_Pur...|          5|           30|         30|   N|                Y|Excellent Jazz album|Excellent album. ...| 2015-08-31|
|         US|   16227164| RCDI7Z8VCZJI9|B00YHTO1HS|     771772746|Badlands (Deluxe)...|Digital_Music_Pur

In [8]:
# Total Count of reviews 
TotCount = df.count()
print(f"There are {TotCount} reviews.")

# Total Count of 5 star reviews
Count_5star = df.filter(df["star_rating"]== 5).count()
print(f"There are {Count_5star} 5 star reviews.")

There are 1688884 reviews.
There are 1345146 5 star reviews.


In [9]:
# Creating DataFrame for 5 star reviews
five_star = df.filter(df["star_rating"]== 5)

In [10]:
# percentage of five star paid reviews
FiveStarPaid = five_star.filter(five_star["vine"] == "Y").count()/df.count()
print(f"The percentage of 5 star reviews is {FiveStarPaid}.")

# percentage of five start unpaid reviews
FiveStarUnP= five_star.filter(five_star["vine"] == "N").count()/df.count()
print(f"The percentage of non 5 star reviews is {FiveStarUnP}.")

The percentage of 5 star reviews is 0.0.
The percentage of non 5 star reviews is 0.7964703318878028.


In [11]:
# Deliverable 3

In [12]:
# How many Vine reviews and non-Vine reviews were there?
Vine = df.filter(df["vine"] == "Y").count()
print(f"There are {Vine} Vine reviews.")

NonVine = df.filter(df["vine"] == "N").count()
print(f"There are {NonVine} non-Vine reviews.")

There are 0 Vine reviews.
There are 1688881 non-Vine reviews.


In [13]:
# How many Vine reviews were 5 stars? How many non-Vine reviews were 5 stars?
Non5Vine = df.filter((df["vine"] == "N") & (df["star_rating"] == 5)).count()
print(f"There are {Non5Vine} non-Vine 5 star reviews.")

There are 1345146 non-Vine 5 star reviews.


In [14]:
# What percentage of Vine reviews were 5 stars? What percentage of non-Vine reviews were 5 stars?
NonVine5StarPer = Non5Vine/TotCount
print(f"The percentage of non-Vine 5 Star reviews is {NonVine5StarPer}%.")

The percentage of non-Vine 5 Star reviews is 0.7964703318878028%.
